# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
# Pull csv from Part I
city_data = pd.read_csv('city_data.csv')
city_data.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_data[['Lat', 'Lng']]

# Fill NaN values and convert to float
humidity = city_data['Humidity'].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Drop cities where:
# Max Temp is lower than 70
# Max Temp is greater than 80
# Wind speed is over 10mph
# Cloudiness is greater than 0

unfit_city = city_data[(city_data['Max Temp'] < 70) |
                      (city_data['Max Temp'] > 80) |
                      (city_data['Wind Speed'] > 10) |
                      (city_data['Cloudiness'] > 0)].index

city_data.drop(unfit_city, inplace=True)

city_data

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Drop column 'Unnamed: 0'
hotel_df = city_data.drop('Unnamed: 0', axis=1)

# Add column 'Hotel Name'
hotel_df['Hotel Name'] = ''

hotel_df

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": "hotel",
    "type": "lodging",
    "radius": 5000,
    "key": g_key
}

In [ ]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get coordinates from df
    lat = row['Lat']
    lng = row['Lng']
    
    city = row['City']
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
             
    except (KeyError, IndexError):
        print("Can't find result... skipping.")
        
    print("------------")

In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))